# Some more SQL, and Database modeling

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///lect03.sqlite

## Short recapitulation

Last time we talked about _inner joins_, in our examples we
had three tables:

In [ ]:
%%sql
SELECT *
FROM   students
LIMIT  4

In [ ]:
%%sql
SELECT *
FROM   colleges
LIMIT  4

In [ ]:
%%sql
SELECT *
FROM   applications
LIMIT  4

**Exercise:** _Show the names of all students who have
applied for a college in California -- also show the college
and major._

In [ ]:
%%sql


## Outer joins

An inner join combines rows in different tables _when there
is a match in the other table_, rows with no corresponding
row in the other table will not turn up in the joined table.

With an _outer join_ we can make sure that every row in one
or both of the tables turn up in the joined table -- in case
there is no match, it will be paired with a row containing
unly `NULL` values.

As as example, we've seen how to join students and their
applications:

In [ ]:
%%sql
SELECT s_name, s_id, c_name, major
FROM   students
JOIN   applications
USING  (s_id)

Now, assume we also want to see those students who haven't
applied, we can do that using a _left outer join_:

In [ ]:
%%sql
SELECT          s_name, s_id, c_name, major
FROM            students
LEFT OUTER JOIN applications
USING           (s_id)

We still get the rows we got before, but now we also get
rows with `NULL`'s in the attributes from the right
relation, if there is no row in the right table which
corresponds to a row in the left table (i.e., the `s_id`
found in the `students` table has no match in the
`applications` table -- the tuples on the left are sometimes
called _dangling tuples_).

**Question:** _How can we use the `NULL`-padded rows from a
left outer join?_ (there's an answer at the end)

By the way, we can omit the `OUTER` keyword:

In [ ]:
%%sql
SELECT    s_name, s_id, c_name, major
FROM      students
LEFT JOIN applications
USING     (s_id)

There is a 'natural' version of outer joins ...

In [ ]:
%%sql
SELECT            s_name, s_id, c_name, major
FROM              students
NATURAL LEFT JOIN applications

... but for the same reasons as for inner joins, it's much
better to explicitly declare which attribute we're joining
(using `USING`).

There is a way to get the same result without inner or outer
joins, but it requires more code:

In [ ]:
%%sql
SELECT s_name, s.s_id, c_name, major
FROM   students s, applications a
WHERE  s.s_id = a.s_id
UNION
SELECT s_name, s_id, NULL, NULL
FROM   students
WHERE  s_id NOT IN (SELECT s_id
                    FROM   applications)

Seing this might help you understand what left outer join
actually returns.

Now, the opposite problem, we want to see applications with
no matching students -- of course we could just swap
`students` and `applications` in the query above, but we
could also use a _right outer join_:

In [ ]:
%%sql
SELECT     s_name, s_id, c_name, major
FROM       students
RIGHT JOIN applications USING (s_id)

Right outer joins are not supported in SQLite (yet), though.

There is also a _full outer join_, which combines the left-
and the right outer joins:

In [ ]:
%%sql
SELECT    s_name, s_id, c_name, major
FROM      students
FULL JOIN applications USING (s_id)

Of the outer joins, SQLite only supports left outer, the
other ones are left out :-).

**Answer to the question above:** We can use outer joins to
find missing values, e.g., we can find the names of all
studens who haven't applied:

In [ ]:
%%sql
SELECT    s_id, s_name
FROM      students
LEFT JOIN applications
USING     (s_id)
WHERE     c_name IS NULL

We do an outer join, and see where the result is padded with
`NULL`'s (observe that we have to test it with "`IS NULL`").

**Exercise:** _Find all 'spurious' applications, i.e.,
applications where the student id doesn't match any
student._ Try to solve the exercises in two ways (one with
an outer join, the other with set operations).

In [ ]:
%%sql


In [ ]:
%%sql


**Exercise:** _Try to generate the unwieldly college
application table we started out with above from the three
smaller tables._

In [ ]:
%%sql


## More subqueries

Another form of subquery is:

```sql
SELECT ...,
       (SELECT ...
        FROM ...
        WHERE ...)
FROM   ...
```


This works if the subquery produces one result, such as when
we use an aggregate function without groups.

We can sometimes use this pattern to perform a kind of
nestled loop over a table, this normally requires us to use
aliases in the `FROM` section -- as an example, solve the
following problem:

**Exercise:** _Output the name of all laureates, and the number of
awards they have -- order first by number of awards, then by
name, and show only the first 10._

In [ ]:
%%sql
SELECT DISTINCT laureate,
       (SELECT COUNT(*)
        FROM   nobel AS inner
        WHERE  inner.laureate = outer.laureate) AS nbr_of_awards
FROM   nobel AS outer
ORDER BY nbr_of_awards DESC, laureate
LIMIT 10

This is called a _correlated subquery_ (since we refer to
the enclosing query inside it). We use aliases to
distinguish between the nobel table in the outer query and
the nobel table in the subquery (it's the same table, but we
'iterate' through it separately).


## Views

If we often use some query as part of other queries, we can
save it using a _view_:

In [ ]:
%%sql
CREATE VIEW named_applications AS
  SELECT s_id, s_name AS name, major, c_name AS college, state
  FROM   applications
  JOIN   students
  USING  (s_id)
  JOIN   colleges
  USING  (c_name);

SELECT DISTINCT name, state
FROM   named_applications
WHERE  s_id IN (123, 234, 456);

**Exercise:** _Show the year and category for everyone who
has won more than one Nobel Prize -- use a view to do it._

In [ ]:
%%sql
CREATE VIEW multiple_awards AS
  SELECT laureate
  FROM   nobel
  GROUP BY laureate
  HAVING COUNT() > 1;

SELECT year, category, laureate
FROM   multiple_awards
JOIN   nobel
USING  (laureate)
ORDER BY year;

A view will be around until someone explicitly drops it:

In [ ]:
DROP VIEW IF EXISTS multiple_awards;

## Using 'local' tables

We can also create one or more 'local tables' in a query,
using it is a way to break up a complicated query into
smaller pieces while still keeping it to one SQL statement.

So instead of creating the view above, we can write:

In [ ]:
%%sql
WITH multiple_awards AS
  (SELECT laureate
   FROM   nobel
   GROUP BY laureate
   HAVING COUNT() > 1)
SELECT year, category, laureate
FROM   multiple_awards
JOIN   nobel
USING  (laureate)
ORDER BY year;

The tables created with `WITH` will disappear as soon as our
query is finished, the point of the `WITH` statement is that
it's one single query.